In [1]:
import numpy as np
import pandas as pd

# To process embeddings
import tensorflow_hub as hub
from langdetect import detect
from pathlib import Path
from nltk.tokenize import sent_tokenize

# To create recommendations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# To create sentence clusters
from sklearn.cluster import KMeans

# To load saved embeddings
import joblib

# To match strings
import re

# To create webapp
import psutil
import streamlit as st
st.set_page_config(page_title='3380 Books',
                  layout="wide",
                  page_icon= ':books:')
from streamlit import caching


# To sort final recommendation list
from collections import Counter

def dataLoader(datapath, books_file, reviews_file, reviewsAll_file):
    '''
    Loads DataFrames with books and review information
    '''
    books = pd.read_csv(datapath + books_file).drop('Unnamed: 0', axis=1).fillna('')
    reviews = pd.read_csv(datapath + reviews_file).drop('Unnamed: 0', axis=1)
    reviewsAll = pd.read_csv(datapath + reviewsAll_file).drop('Unnamed: 0', axis=1)
    return books, reviews, reviewsAll

def loadEmbeddings():
    '''
    Loads pre-trained sentence and review arrays
    '''
    # Path to USE
    embed = hub.load('/media/einhard/Seagate Expansion Drive/3380_data/data/tensorflow_hub/universal-sentence-encoder_4')

    # Load pre-trained sentence arrays
    ## Reviews array is a set of embeddings trained on review lengths of < 90 characters
    reviews_array = joblib.load('/media/einhard/Seagate Expansion Drive/3380_data/data/Models/reviewEmbeddings.pkl')
    ## Descriptions array is a set of embeddings trained on all book descriptions
    descriptions_array = joblib.load('/media/einhard/Seagate Expansion Drive/3380_data/data/Models/descriptionEmbeddings.pkl')

    return embed, reviews_array, descriptions_array

def embedInputs(books_df, review_df, search_param, review_max_len, searchTitle=True):
    '''
    Converts input reviews into USE arrays. Returns vectorized reviews for the book that was
    passed as bookTitle.

    Args:
        search_param = List of book titles or author names whose reviews we want to embed. For authors, see 'searchTitle' argument
        books_df = DataFrame with book_id information
        review_df = DataFrame with book_id and review text
        searchTitle = If True, will search for book_id based on title of a book. If False, it will look for author names to find book_id.
    '''
    if searchTitle:
        #Finds book_id from title of book
        input_book_id = books_df[books_df.title.isin([search_param])].book_id.tolist()
    else:
        # Finds book_id from author name
        input_book_id = books_df[books_df.name.isin([search_param])].book_id.tolist()

    # Finds reviews for specified book
    input_sentences = review_df[review_df.book_id.isin(input_book_id)].review_text

    # Filters review length
    input_sentences = input_sentences[input_sentences.str.len() <= review_max_len]

    # Converts reviews into 512-dimensional arrays
    input_vectors = embed(input_sentences)

    # Returns reviews and vectorized reviews for a particular book
    return input_sentences, input_vectors

def getClusters(input_vectors, n_clusters):
    '''
    Creates KMeans instance and fits model.

    The for nested for loop is used to display the returned sentences on Streamlit.

    Args:
        input_sentences =  Sentences to compare
        n_clusters = How many clusters to generate
    '''
    tsne = TSNE(n_components=3, verbose=1, perplexity=80, n_iter=5000, learning_rate=200)
    TSNE_transformed = tsne.fit_transform(input_vectors)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, algorithm='full')
    return kmeans.fit(TSNE_transformed)


def showClusters(input_sentences, input_vectors, authorTitle, n_clusters, n_results, model, searchTitle=True):
    '''
    This function will find theme clusters in the reviews of a particular book or set of sentences.
    Uses cluster centers to find semantically similar sentences to the input vectors.

    The nested for loop is used to display the returned sentences on Streamlit.

    Args:
        input_sentences =  Sentences to compare
        input_vectors = USE Array generated by embedding input sentences
        authorTitle = Title of book in question, or name of author --> Used to display header only
        n_clusters = How many clusters to generate
        n_results = How many sentences to display per n_cluster
        model = The model used to create the clusters.
    '''
    if searchTitle:
        # Displays which book's reviews are being clustered
        print(f'Opinion clusters about *{authorTitle}*')
    else:
        print(f'Opinion clusters about {authorTitle}\'s books')

    # Iterates through centroids to and computes inner products to find nlargest
    for i in range(n_clusters):
        centre = model.cluster_centers_[i]
        inner_product = np.inner(centre, input_vectors)
        indices = pd.Series(inner_product).nlargest(n_results).index
        clusteredInputs = list(input_sentences.iloc[indices])

        # Prints reviews that are closest to centroid
    
        print(f'**Cluster #{i+1}**')
        for sentence in clusteredInputs:
            print(sentence)
        return indices, inner_product

def cleanAndTokenize(df, filepath, searchTitle, author):
    if searchTitle:
        if Path(filepath + 'app_data/book_id/' + df.book_id.iloc[1].astype(str) + '.csv').is_file():
            sentences_df = pd.read_csv(filepath + 'app_data/book_id/' + df.book_id.iloc[1].astype(str) + '.csv').drop('Unnamed: 0', axis=1)
        else:
            reviews_df = clean_reviews(df)
            sentences_df =  make_sentences(reviews_df)
            sentences_df.to_csv(filepath + 'app_data/book_id/' + df.book_id.iloc[1].astype(str) + '.csv')
        sentences_df.book_id = sentences_df.book_id.astype(int)
        return sentences_df

    else:
        if Path(filepath + 'app_data/author/' + author + '.csv').is_file():
            sentences_df = pd.read_csv(filepath + 'app_data/author/' + author + '.csv').drop('Unnamed: 0', axis=1)
        else:
            reviews_df = clean_reviews(df)
            sentences_df =  make_sentences(reviews_df)
            sentences_df.to_csv(filepath + 'app_data/author/' + author + '.csv')
        sentences_df.book_id = sentences_df.book_id.astype(int)

        return sentences_df
def clean_reviews(df):
    '''
    Copyright (c) 2020 Willie Costello
    '''
    # Drop duplicates
    df.drop_duplicates(inplace=True)

    # Define spoiler marker & remove from all reviews
    spoiler_str_ucsd = '\*\* spoiler alert \*\* \n'
    df['review_text'] = df['review_text'].str.replace(spoiler_str_ucsd, '')

    # Replace all new line characters
    df['review_text'] = df['review_text'].str.replace('\n', ' ')

    # Append space to all sentence end characters
    df['review_text'] = df['review_text'].str.replace('.', '. ').replace('!', '! ').replace('?', '? ')

    # Initialize dataframe to store English-language reviews
    reviews_df = pd.DataFrame()

    # Loop through each row in dataframe
    for i in range(len(df)):

        # Save review to variable
        review = df.iloc[i]['review_text']

        # Check if review is English
        if detect(review) == 'en':
            # If so, add row to English-language dataframe
            reviews_df = reviews_df.append(df.iloc[i, :])

    reviews_df.book_id = reviews_df.book_id.astype(int)
    return reviews_df


def make_sentences(reviews_df):
    '''
    Copyright (c) 2020 Willie Costello
    '''
    # Initialize dataframe to store review sentences, and counter
    sentences_df = pd.DataFrame()
    ctr = 0


    # Loop through each review
    for i in range(len(reviews_df)):

        # Save row and review to variables
        row = reviews_df.iloc[i]
        review = row.loc['review_text']

        # Tokenize review into sentences
        sentences = sent_tokenize(review)

        # Loop through each sentence in list of tokenized sentences
        for sentence in sentences:
            # Add row for sentence to sentences dataframe
            new_row = row.copy()
            new_row.at['review_text'] = sentence
            sentences_df = sentences_df.append(new_row, ignore_index=True)
        ctr += 1
    sentences_df = sentences_df[(sentences_df.review_text.str.len() >= 20) & (sentences_df.review_text.str.len() <= 350)]
    return sentences_df

def embedInputs(books_df, review_df, search_param, review_max_len, searchTitle=True):
    '''
    Converts input reviews into USE arrays. Returns vectorized reviews for the book that was
    passed as bookTitle.

    Args:
        search_param = List of book titles or author names whose reviews we want to embed. For authors, see 'searchTitle' argument
        books_df = DataFrame with book_id information
        review_df = DataFrame with book_id and review text
        searchTitle = If True, will search for book_id based on title of a book. If False, it will look for author names to find book_id.
    '''
    if searchTitle:
        #Finds book_id from title of book
        input_book_id = books_df[books_df.title.isin([search_param])].book_id.tolist()
        author_name = 'VariousAuthors/'
    else:
        # Finds book_id from author name
        input_book_id = books_df[books_df.name.isin([search_param])].book_id.tolist()
        author_name = books_df[books_df.name.isin([search_param])].name.iloc[0]

    # Finds reviews for specified book
    # input_sentences = review_df[review_df.book_id.isin(input_book_id)].review_text
    input_sentences = cleanAndTokenize(review_df[review_df.book_id.isin(input_book_id)], tokenizedData, searchTitle=searchTitle, author=author_name).review_text

    # Filters review length
    input_sentences = input_sentences[input_sentences.str.len() <= review_max_len]

    # Converts reviews into 512-dimensional arrays
    input_vectors = embed(input_sentences)

    # Returns reviews and vectorized reviews for a particular book
    return input_sentences, input_vectors

In [40]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [41]:
model = getClusters(input_vectors, 8)
indices, inner_product = showClusters(input_sentences, input_vectors, 'Children of Time', 8, 8, model, searchTitle=True)

[t-SNE] Computing 141 nearest neighbors...
[t-SNE] Indexed 142 samples in 0.001s...
[t-SNE] Computed neighbors for 142 samples in 0.007s...
[t-SNE] Computed conditional probabilities for sample 142 / 142
[t-SNE] Mean sigma: 0.474998
[t-SNE] KL divergence after 250 iterations with early exaggeration: 131.341644
[t-SNE] KL divergence after 5000 iterations: 1.313684
Opinion clusters about *Children of Time*


ValueError: shapes (3,) and (512,142) not aligned: 3 (dim 0) != 512 (dim 0)

In [52]:
inner_product.argmin()

287

In [2]:
# Paths to books and reviews DataFrames
datapath = '/media/einhard/Seagate Expansion Drive/3380_data/data/'

# Stores tokenized reviews so they only need to be processed the first time that particular book is called
tokenizedData = '/media/einhard/Seagate Expansion Drive/3380_data/data/'
books_file = 'Filtered books/clean_filtered_books.csv'
reviews_file = 'Filtered books/clean_filtered_reviews.csv'
reviewsAll_file = 'Filtered books/reviews_for_cluster.csv'

# Loading DataFrames
books, reviews, reviewsAll = dataLoader(datapath, books_file, reviews_file, reviewsAll_file)

# Loadding pre-trained embeddings and embedder for input sentences
embed, reviews_array, descriptions_array = loadEmbeddings()

# Setting base URL for Goodreads
goodreadsURL = 'https://www.goodreads.com/book/show/'

2020-11-30 22:57:16.485 INFO    absl: resolver HttpCompressedFileResolver does not support the provided handle.
2020-11-30 22:57:16.486 INFO    absl: resolver GcsCompressedFileResolver does not support the provided handle.
2020-11-30 22:57:16.487 INFO    absl: resolver HttpUncompressedFileResolver does not support the provided handle.


In [6]:
books.sample(1).title.tolist()

['Hope for the Flowers']

# Testing full pipeline

In [26]:
import tensorflow_hub as hub
from langdetect import detect
from pathlib import Path
from nltk.tokenize import sent_tokenize

In [4]:
input_text = 'Children of Time'
book_title = books[books.title.str.contains(input_text, case=False)].title.tolist()
input_book_id = books[books.title.isin(book_title)].book_id.tolist()

In [5]:
input_book_id

[25499718]

In [6]:
reviewsAll[reviewsAll.book_id.isin([input_book_id])]



input_sentences = cleanAndTokenize(reviewsAll[reviewsAll.book_id.isin(input_book_id)], tokenizedData, searchTitle=True, author='Adrian Tchaikovsky').review_text

# Converts reviews into 512-dimensional arrays
input_vectors = embed(input_sentences)
# Returns reviews and vectorized reviews for a particular book

In [21]:
input_sentences

0                                 Really liked this one.
1                Fantastic and original science fiction.
2                            Setting the stage for more.
3                            I really enjoyed this book.
4      I read a lot of sci-fi and this was up there w...
                             ...                        
137    Just about the right balance between science a...
138    Yesterday I had to break a tiny web that had a...
139                    "I'm sorry, Portia," I whispered.
140    I thought the book dragged a bit in the middle...
141         It just pushed all the right buttons for me.
Name: review_text, Length: 142, dtype: object

In [22]:
input_vectors

<tf.Tensor: shape=(142, 512), dtype=float32, numpy=
array([[-0.03793607,  0.04623649, -0.00505508, ..., -0.04547682,
         0.00827182, -0.00052418],
       [-0.04669672,  0.03274052, -0.07346593, ..., -0.04710089,
         0.09411614,  0.04661921],
       [ 0.02117739, -0.02780496,  0.01408575, ..., -0.00671999,
         0.07057461,  0.01192718],
       ...,
       [-0.02475809,  0.023139  ,  0.006002  , ...,  0.00268129,
         0.08098436,  0.0375176 ],
       [-0.04929401, -0.05269561,  0.05049564, ..., -0.03259405,
         0.11280895,  0.03305841],
       [-0.0424552 ,  0.00778071,  0.0133794 , ..., -0.00455391,
        -0.00720548, -0.03733094]], dtype=float32)>

Opinion clusters about *Children of Time*
EUCLEDIAN DISTANCES 
 [[0.97986721 0.6898791  1.17845609 ... 1.13109286 0.93589187 1.15141068]
 [1.06466306 1.05215896 1.15893397 ... 1.10636515 1.01618467 1.10651268]
 [0.81628725 0.96221752 1.19189276 ... 1.15539174 0.95476636 1.13788915]
 ...
 [0.95494688 1.01326106 1.16443593 ... 1.15406938 0.87793586 1.14085719]
 [1.05721386 0.80401717 1.07956127 ... 1.11902202 1.06704623 1.12727826]
 [0.91293835 0.97519208 1.03292782 ... 1.05985123 0.82227564 0.91017246]]


Exception: Data must be 1-dimensional

In [25]:
input_sentences, input_vectors = embedInputs(books,
                                            reviewsAll,
                                            search_param=info.title.tolist()[0],
                                            review_max_len=review_max_len,
                                            searchTitle=True)

NameError: name 'info' is not defined

In [27]:
from sklearn.metrics.pairwise import euclidean_distances

# Tokenizing reviews into sentences

In [3]:
from nltk.tokenize import sent_tokenizea

In [15]:
search_param = books[books.title == 'Children of Time'].name.tolist()[0]
books[books.name.isin([search_param])].name.iloc[0]

'Adrian Tchaikovsky'

In [5]:
books[books.title.str.contains("Harry Potter and the Sorcerer's Stone")]

,title,description,book_id,weighted_score,name
309,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter's life is miserable. His parents ...,3,4.449994,J.K. Rowling
329,Harry Potter and the Sorcerer's Stone (Harry P...,An alternative cover for this ASIN can be foun...,28132722,4.448603,J.K. Rowling
342,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter thinks he is an ordinary boy - un...,13562891,4.446682,J.K. Rowling


In [8]:
reviewsAll[reviewsAll.book_id.isin([3])]

,book_id,review_text
169,3,Tuve el gusto de leerlo antes de que saliera l...
214,3,I remember trying 3 times to read this but I a...
1246,3,this was not the best harry potter book
1793,3,"I love the Harry Potter series, even though th..."
4229,3,One of the best and most magical children's bo...
...,...,...
1391054,3,decent wizarding book sequels are better
1391095,3,"Just started this, but a little hesitant to ge..."
1391392,3,Good...not as exciting as the others though...
1391659,3,The start of Harry's adventure in Hogwarts. My...


# Keeping only English

In [92]:
from langdetect import detect
from pathlib import Path

In [21]:
def clean_reviews(df):

    # Read in CSV as dataframe
    length_orig = len(df)

    # Drop duplicates
    df.drop_duplicates(inplace=True)
    num_dups = length_orig - len(df)

    print(f'Read in {length_orig} reviews, dropping {num_dups} duplicates\n')

    # Define spoiler marker & remove from all reviews
    spoiler_str_ucsd = '\*\* spoiler alert \*\* \n'
    df['review_text'] = df['review_text'].str.replace(spoiler_str_ucsd, '')

    # Replace all new line characters
    df['review_text'] = df['review_text'].str.replace('\n', ' ')

    # Append space to all sentence end characters
    df['review_text'] = df['review_text'].str.replace('.', '. ').replace('!', '! ').replace('?', '? ')

    # Initialize dataframe to store English-language reviews
    reviews_df = pd.DataFrame()
    # Initialize counter for dropped reviews
    drop_ctr = 0

    # Loop through each row in dataframe
    for i in range(len(df)):

        # Save review to variable
        review = df.iloc[i]['review_text']

        # Check if review is English
        try:
            if detect(review) == 'en':
                # If so, add row to English-language dataframe
                reviews_df = reviews_df.append(df.iloc[i, :])
            else:
                # If not, add 1 to dropped review counter
                drop_ctr += 1
        # If check fails, add 1 to dropped review counter
        except:
            drop_ctr += 1

    reviews_df.book_id = reviews_df.book_id.astype(int)

    print(f'Dropped {drop_ctr} non-English reviews. '
          f'{len(reviews_df)} reviews remain.\n')

    return reviews_df

In [67]:
clean_reviews(reviewsAll[reviewsAll.book_id.isin([3])], 30, 100)

Read in 3562 reviews, dropping 0 duplicates

Dropped 189 non-English reviews. 1480 reviews remain.



,book_id,review_text
1246,3,this was not the best harry potter book
5361,3,The beginning of an inspiring literary traditi...
5573,3,So much love for JK Rowling and her harry pott...
6964,3,"An excellent book, great for kids, not sure I'..."
7329,3,Love it as well as all the Harry Potter series.
...,...,...
1389325,3,I love the Harry Potter series.
1390184,3,I enjoyed Reading this book again. Will conti...
1390717,3,a beautiful adventure wanna be with harry po...
1390777,3,Sums up a child's experiences going into the r...


In [22]:
def make_sentences(reviews_df):
    '''
    Copyright (c) 2020 Willie Costello
    '''
    # Initialize dataframe to store review sentences, and counter
    sentences_df = pd.DataFrame()
    ctr = 0

    print(f'Starting tokenization')

    # Loop through each review
    for i in range(len(reviews_df)):

        # Save row and review to variables
        row = reviews_df.iloc[i]
        review = row.loc['review_text']

        # Tokenize review into sentences
        sentences = sent_tokenize(review)

        # Loop through each sentence in list of tokenized sentences
        for sentence in sentences:
            # Add row for sentence to sentences dataframe
            new_row = row.copy()
            new_row.at['review_text'] = sentence
            sentences_df = sentences_df.append(new_row, ignore_index=True)

        ctr += 1
        if (ctr % 500 == 0):
            print(f'{ctr} reviews tokenized')

    sentences_df = sentences_df[(sentences_df.review_text.str.len() > 20) & (sentences_df.review_text.str.len() < 160)]
    print(f'Tokenization complete: {len(sentences_df)} sentences tokenized\n')

    return sentences_df

In [23]:
tokenizedData = '/media/einhard/Seagate Expansion Drive/3380_data/data/app_data/'
def cleanAndTokenize(df, filepath):
    if Path(filepath + df.book_id.iloc[1].astype(str) + '.csv').is_file():
        sentences_df = pd.read_csv(filepath + df.book_id.iloc[1].astype(str) + '.csv').drop('Unnamed: 0', axis=1)
    else:
        reviews_df = clean_reviews(df)
        sentences_df =  make_sentences(reviews_df)
        sentences_df.to_csv(filepath + df.book_id.iloc[1].astype(str) + '.csv')
    sentences_df.book_id = sentences_df.book_id.astype(int)
    return sentences_df


In [105]:
cleanAndTokenize(reviewsAll[reviewsAll.book_id.isin([3])], filepath)

,book_id,review_text
0,3,I remember trying 3 times to read this but I a...
1,3,So I skipped to book 2 and then read the whole...
2,3,It's hard to review this book without taking i...
3,3,this was not the best harry potter book
4,3,"I love the Harry Potter series, even though th..."
...,...,...
6915,3,I love the movies and am looking forward to ge...
6916,3,Twi-hard/Potterhead in the making baby!
6917,3,not as exciting as the others though.
6918,3,The start of Harry's adventure in Hogwarts.


In [76]:
import time
startTime = time.time()
reviewsCleaned = clean_reviews(reviewsAll[reviewsAll.book_id.isin([3])])
reviewsTokenized = make_sentences(reviewsCleaned, 20, 160)


reviewsEmbedded = embed(reviewsTokenized.review_text)
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Read in 3562 reviews, dropping 0 duplicates

Dropped 187 non-English reviews. 3375 reviews remain.

Starting tokenization
500 reviews tokenized
1000 reviews tokenized
1500 reviews tokenized
2000 reviews tokenized
2500 reviews tokenized
3000 reviews tokenized
Tokenization complete: 7802 sentences tokenized

Execution time in seconds: 39.682156562805176


In [88]:
print(filepath + reviewsAll[reviewsAll.book_id.isin([3])].book_id.iloc[1].astype(str) + '.csv')

/media/einhard/Seagate Expansion Drive/3380_data/data/app_data/3.csv


In [55]:
reviewsTokenized.book_id = reviewsTokenized.book_id.astype(int)

In [87]:
reviewsAll[reviewsAll.book_id.isin([3])].book_id.iloc[1].astype(str)

'3'

["Harry Potter and the Sorcerer's Stone (Harry Potter, #1)"]

In [49]:
startTime = time.time()
embed(reviewsTokenized.review_text)
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 0.8115620613098145


# Testing spaCy

In [46]:
from spacy.pipeline import Sentencizer

In [45]:
reviewsTokenized

,book_id,review_text
0,3.0,Tuve el gusto de leerlo antes de que saliera l...
1,3.0,I remember trying 3 times to read this but I a...
2,3.0,So I skipped to book 2 and then read the whole...
3,3.0,It's hard to review this book without taking i...
4,3.0,this was not the best harry potter book
...,...,...
8625,3.0,Good...not as exciting as the others though...
8626,3.0,The start of Harry's adventure in Hogwarts.
8627,3.0,My most favorite book in the series!
8628,3.0,WINGARDIUM LEVIOSA!


In [ ]:
reviewsTokenized.to_csv('/media/einhard/Seagate Expansion Drive/3380_data/data/tokenized sentences/reviewsTokenized.csv')

# Experimental

In [3]:
import numpy as np
import pandas as pd

# To process embeddings
import tensorflow_hub as hub
from langdetect import detect
from pathlib import Path
from nltk.tokenize import sent_tokenize

# To create recommendations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# To create sentence clusters
from sklearn.cluster import KMeans

# To load saved embeddings
import joblib

# To match strings
import re

# To create webapp
import psutil

In [4]:
# Experimental
def findSemanticallySimilarReviews(query,reviews, books, sentence_array,  n_books):
    # Create vector from query and compare with global embedding
    sentence = [query]
    sentence_vector = np.array(embed(sentence))
    inner_product = np.inner(sentence_vector, sentence_array)[0]

    # Find sentences with highest inner products
    top_n_sentences = pd.Series(inner_product).nlargest(n_books+1)
    top_n_indices = top_n_sentences.index.tolist()
    review_index = reviews.iloc[top_n_indices].index

    books_beta = books[books.book_id.isin(reviews.iloc[review_index].book_id)].sort_values('book_id')
    reviews_beta = reviews.iloc[review_index].sort_values('book_id')

    return books_beta, reviews_beta

In [5]:
n_result = 9

In [6]:
books_, reviews_ = findSemanticallySimilarReviews(query='Charlemagne',
                                               reviews=reviewsAll,
                                               books=books,
                                               n_books=10,
                                               sentence_array=reviews_array)

In [7]:
reviews_

,book_id,review_text
211406,28187,A reread. (I'm going to be rereading this seri...
146200,34498,"listening as always, stephen briggs is amazing"
324717,77203,This is a very emotional read. It was difficul...
420175,99561,"Great book! Very well written, so many good li..."
99869,110392,I liked Dunford I even liked Henry but the rus...
186002,7260188,I liked the first two books of the trilogy bet...
124971,7992995,Enjoyed the book despite the cover. Definitely...
40079,12218678,I did not care for this novel. I lost all resp...
167426,12408238,I needed a soothing book after reading Lords o...
360406,15717876,Tried a sample on amazon. It was too corny and...


# Topic Modelling

In [8]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', ])


In [12]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)

def process_words(texts,bigram_mod, trigram_mod, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

def format_topics_sentences(corpus, texts, ldamodel=None,):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

def findTopic(reviewCluster):

    # Convert to list
    #data = reviewCluster.values.tolist()
    data_words = list(sent_to_words(reviewCluster))
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    data_ready = process_words(data_words, bigram_mod=bigram_mod,trigram_mod=trigram_mod)
    id2word = corpora.Dictionary(data_ready)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in data_ready]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=4, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=10,
                                               passes=10,
                                               alpha='symmetric',
                                               iterations=100,
                                               per_word_topics=True)
    df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


    sent_topics_sorteddf_mallet = pd.DataFrame()
    sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

    for i, grp in sent_topics_outdf_grpd:
        sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                                 grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                                axis=0)

    # Reset Index    
    sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

    # Format
    sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

    # Show
    return sent_topics_sorteddf_mallet['Representative Text'].tolist()[0]

In [13]:
findTopic(reviews_['review_text'])

['enjoy',
 'book',
 'cover',
 'definitely',
 'triumph',
 'evil',
 'theme',
 'like',
 'little',
 'strong',
 'female',
 'heroine']